In [172]:
from datetime import datetime, timedelta
import glob
from zipfile import ZipFile
import os
import shutil
import pandas as pd
import numpy as np

# Reorganize data

In [2]:
option_data_folder = 'E:\projects\stock data'

In [3]:
shutil.rmtree(os.path.join(option_data_folder, 'fixed'))

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'E:\\projects\\stock data\\fixed\\2015.zip'

In [4]:
all_fnames = list(glob.glob(f'{option_data_folder}/*'))
for year in range(2005,2021):
    print(f'------------ {year} ----------------')
    folder = os.path.join(option_data_folder, 'data', str(year))
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    # extrtact all files into a single, temp, directory
    files = [f for f in all_fnames if str(year) in f]
    for fname in files:
        print(f'extracting {fname}')
        with ZipFile(fname) as zip_file:
            for member in zip_file.namelist():
                filename = os.path.basename(member)
                # skip directories
                if not filename:
                    continue

                df = pd.read_csv(zip_file.open(member))
                
                if bad_stocks is not None:
                    print('removing bad stocks')
                    df = df[df['']]
                    
                strike_ratio = df['StrikePrice'] / df['UnderlyingPrice']
                df = df[(0.5 < strike_ratio) & (strike_ratio < 2)]
                df = df[df['Volume'] > 0]
                df[df['UnderlyingPrice'] > 1.0]
                df.to_csv(os.path.join(folder, filename))

------------ 2005 ----------------
extracting E:\projects\stock data\2005.zip
------------ 2006 ----------------
extracting E:\projects\stock data\2006.zip
------------ 2007 ----------------
extracting E:\projects\stock data\2007.zip
------------ 2008 ----------------
extracting E:\projects\stock data\2008.zip
------------ 2009 ----------------
extracting E:\projects\stock data\2009.zip
------------ 2010 ----------------
extracting E:\projects\stock data\2010.zip
------------ 2011 ----------------
extracting E:\projects\stock data\2011.zip
------------ 2012 ----------------
extracting E:\projects\stock data\2012.zip
------------ 2013 ----------------
extracting E:\projects\stock data\2013_1.zip
extracting E:\projects\stock data\2013_2.zip
------------ 2014 ----------------
extracting E:\projects\stock data\2014_1.zip
extracting E:\projects\stock data\2014_2.zip
------------ 2015 ----------------
extracting E:\projects\stock data\2015_1.zip
extracting E:\projects\stock data\2015_2.zip
e

# Filter options file

In [154]:
symbols = {'SPY'}
start_dt = '20180501'
end_dt = '20190501'
max_strike = 1.1 # relative to underlying price
min_strike = 1 # relative to underlying price
option_types = {'call'} # 'put', 'call'
max_expiration_days = 30
out_path = os.path.join(option_data_folder, 'spy_2018-2019_options.csv')


In [145]:
def filter_file(df):
    ddf = df
    ddf = ddf[ddf['Symbol'].isin(symbols)]
    ddf = ddf[(ddf['StrikePrice'] / ddf['UnderlyingPrice'] <= max_strike) & (ddf['StrikePrice'] / ddf['UnderlyingPrice'] >= min_strike)]
    ddf = ddf[ddf['PutCall'].isin(option_types)]
    ddf = ddf[(ddf['ExpirationDate'] - ddf['DataDate']).dt.days < max_expiration_days]
    return ddf

In [152]:
d = os.path.join(option_data_folder, 'data')
all_fnames = list(glob.glob(f'{d}/*/*'))

df_parts = []
for fname in all_fnames:
    dt = fname[fname.rfind('\\')+1:fname.rfind('_')]
    if dt < start_dt or dt > end_dt:
        continue
    print(f'reading {fname}')
    df =  pd.read_csv(fname, parse_dates=['ExpirationDate', 'DataDate'])
    if len(df) == 0:
        continue
    df_parts.append(filter_file(df))
df = pd.concat(df_parts)

reading E:\projects\stock data\data\2018\20180501_OData.csv
reading E:\projects\stock data\data\2018\20180502_OData.csv
reading E:\projects\stock data\data\2018\20180503_OData.csv
reading E:\projects\stock data\data\2018\20180504_OData.csv
reading E:\projects\stock data\data\2018\20180507_OData.csv
reading E:\projects\stock data\data\2018\20180508_OData.csv
reading E:\projects\stock data\data\2018\20180509_OData.csv
reading E:\projects\stock data\data\2018\20180510_OData.csv
reading E:\projects\stock data\data\2018\20180511_OData.csv
reading E:\projects\stock data\data\2018\20180514_OData.csv
reading E:\projects\stock data\data\2018\20180515_OData.csv
reading E:\projects\stock data\data\2018\20180516_OData.csv
reading E:\projects\stock data\data\2018\20180517_OData.csv
reading E:\projects\stock data\data\2018\20180518_OData.csv
reading E:\projects\stock data\data\2018\20180521_OData.csv
reading E:\projects\stock data\data\2018\20180522_OData.csv
reading E:\projects\stock data\data\2018

reading E:\projects\stock data\data\2018\20181113_OData.csv
reading E:\projects\stock data\data\2018\20181114_OData.csv
reading E:\projects\stock data\data\2018\20181115_OData.csv
reading E:\projects\stock data\data\2018\20181116_OData.csv
reading E:\projects\stock data\data\2018\20181119_OData.csv
reading E:\projects\stock data\data\2018\20181120_OData.csv
reading E:\projects\stock data\data\2018\20181121_OData.csv
reading E:\projects\stock data\data\2018\20181123_OData.csv
reading E:\projects\stock data\data\2018\20181126_OData.csv
reading E:\projects\stock data\data\2018\20181127_OData.csv
reading E:\projects\stock data\data\2018\20181128_OData.csv
reading E:\projects\stock data\data\2018\20181129_OData.csv
reading E:\projects\stock data\data\2018\20181130_OData.csv
reading E:\projects\stock data\data\2018\20181203_OData.csv
reading E:\projects\stock data\data\2018\20181204_OData.csv
reading E:\projects\stock data\data\2018\20181206_OData.csv
reading E:\projects\stock data\data\2018

In [155]:
df.to_csv(out_path)

# Stock prices

In [3]:
df_parts = []
all_fnames = list(glob.glob(f'{option_data_folder}/*'))
for year in range(2005,2021):
    print(f'------------ {year} ----------------')
    folder = os.path.join(option_data_folder, 'data', str(year))
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    # extrtact all files into a single, temp, directory
    files = [f for f in all_fnames if str(year) in f]
    for fname in files:
        with ZipFile(fname) as zip_file:
            for member in zip_file.namelist():
                filename = os.path.basename(member)
                # skip directories
                if not filename:
                    continue

                print(f'reading {fname} : {member}')
                df = pd.read_csv(zip_file.open(member))
                
                df = df.drop_duplicates(['Symbol', 'DataDate'])
                df = df[['Symbol', 'UnderlyingPrice', 'DataDate']].rename(columns={'UnderlyingPrice': 'Price'})
                print(len(df))
                df_parts.append(df)


------------ 2005 ----------------
reading E:\projects\stock data\2005.zip : 2005/2005April/20050401_OData.csv
2514
reading E:\projects\stock data\2005.zip : 2005/2005April/20050404_OData.csv
2515
reading E:\projects\stock data\2005.zip : 2005/2005April/20050405_OData.csv
2513
reading E:\projects\stock data\2005.zip : 2005/2005April/20050406_OData.csv
2513
reading E:\projects\stock data\2005.zip : 2005/2005April/20050407_OData.csv
2509
reading E:\projects\stock data\2005.zip : 2005/2005April/20050408_OData.csv
2509
reading E:\projects\stock data\2005.zip : 2005/2005April/20050411_OData.csv
2515
reading E:\projects\stock data\2005.zip : 2005/2005April/20050412_OData.csv
2514
reading E:\projects\stock data\2005.zip : 2005/2005April/20050413_OData.csv
2515
reading E:\projects\stock data\2005.zip : 2005/2005April/20050414_OData.csv
2515
reading E:\projects\stock data\2005.zip : 2005/2005April/20050415_OData.csv
2515
reading E:\projects\stock data\2005.zip : 2005/2005April/20050418_OData.cs

2463
reading E:\projects\stock data\2005.zip : 2005/2005January/20050125_OData.csv
2465
reading E:\projects\stock data\2005.zip : 2005/2005January/20050126_OData.csv
2464
reading E:\projects\stock data\2005.zip : 2005/2005January/20050127_OData.csv
2466
reading E:\projects\stock data\2005.zip : 2005/2005January/20050128_OData.csv
2466
reading E:\projects\stock data\2005.zip : 2005/2005January/20050131_OData.csv
2471
reading E:\projects\stock data\2005.zip : 2005/2005July/20050701_OData.csv
2627
reading E:\projects\stock data\2005.zip : 2005/2005July/20050705_OData.csv
2628
reading E:\projects\stock data\2005.zip : 2005/2005July/20050706_OData.csv
2631
reading E:\projects\stock data\2005.zip : 2005/2005July/20050707_OData.csv
2630
reading E:\projects\stock data\2005.zip : 2005/2005July/20050708_OData.csv
2645
reading E:\projects\stock data\2005.zip : 2005/2005July/20050711_OData.csv
2646
reading E:\projects\stock data\2005.zip : 2005/2005July/20050712_OData.csv
2649
reading E:\projects\

2721
reading E:\projects\stock data\2005.zip : 2005/2005November/20051117_OData.csv
2720
reading E:\projects\stock data\2005.zip : 2005/2005November/20051118_OData.csv
2723
reading E:\projects\stock data\2005.zip : 2005/2005November/20051121_OData.csv
2715
reading E:\projects\stock data\2005.zip : 2005/2005November/20051122_OData.csv
2720
reading E:\projects\stock data\2005.zip : 2005/2005November/20051123_OData.csv
2721
reading E:\projects\stock data\2005.zip : 2005/2005November/20051125_OData.csv
2721
reading E:\projects\stock data\2005.zip : 2005/2005November/20051128_OData.csv
2721
reading E:\projects\stock data\2005.zip : 2005/2005November/20051129_OData.csv
2723
reading E:\projects\stock data\2005.zip : 2005/2005November/20051130_OData.csv
2724
reading E:\projects\stock data\2005.zip : 2005/2005October/20051003_OData.csv
2694
reading E:\projects\stock data\2005.zip : 2005/2005October/20051004_OData.csv
2692
reading E:\projects\stock data\2005.zip : 2005/2005October/20051005_OData

3021
reading E:\projects\stock data\2006.zip : 2006/2006December/20061211_OData.csv
3025
reading E:\projects\stock data\2006.zip : 2006/2006December/20061212_OData.csv
3026
reading E:\projects\stock data\2006.zip : 2006/2006December/20061213_OData.csv
3030
reading E:\projects\stock data\2006.zip : 2006/2006December/20061214_OData.csv
3029
reading E:\projects\stock data\2006.zip : 2006/2006December/20061215_OData.csv
3032
reading E:\projects\stock data\2006.zip : 2006/2006December/20061218_OData.csv
3016
reading E:\projects\stock data\2006.zip : 2006/2006December/20061219_OData.csv
3019
reading E:\projects\stock data\2006.zip : 2006/2006December/20061220_OData.csv
3022
reading E:\projects\stock data\2006.zip : 2006/2006December/20061221_OData.csv
3024
reading E:\projects\stock data\2006.zip : 2006/2006December/20061222_OData.csv
3026
reading E:\projects\stock data\2006.zip : 2006/2006December/20061226_OData.csv
3026
reading E:\projects\stock data\2006.zip : 2006/2006December/20061227_OD

2751
reading E:\projects\stock data\2006.zip : 2006/2006March/20060308_OData.csv
2753
reading E:\projects\stock data\2006.zip : 2006/2006March/20060309_OData.csv
2760
reading E:\projects\stock data\2006.zip : 2006/2006March/20060310_OData.csv
2761
reading E:\projects\stock data\2006.zip : 2006/2006March/20060313_OData.csv
2761
reading E:\projects\stock data\2006.zip : 2006/2006March/20060314_OData.csv
2760
reading E:\projects\stock data\2006.zip : 2006/2006March/20060315_OData.csv
2761
reading E:\projects\stock data\2006.zip : 2006/2006March/20060316_OData.csv
2768
reading E:\projects\stock data\2006.zip : 2006/2006March/20060317_OData.csv
2771
reading E:\projects\stock data\2006.zip : 2006/2006March/20060320_OData.csv
2765
reading E:\projects\stock data\2006.zip : 2006/2006March/20060321_OData.csv
2773
reading E:\projects\stock data\2006.zip : 2006/2006March/20060322_OData.csv
2773
reading E:\projects\stock data\2006.zip : 2006/2006March/20060323_OData.csv
2776
reading E:\projects\sto

...

4310
reading E:\projects\stock data\2019May.zip : 2019May/20190502_OData.csv
4312
reading E:\projects\stock data\2019May.zip : 2019May/20190503_OData.csv
4305
reading E:\projects\stock data\2019May.zip : 2019May/20190506_OData.csv
4304
reading E:\projects\stock data\2019May.zip : 2019May/20190507_OData.csv
4308
reading E:\projects\stock data\2019May.zip : 2019May/20190508_OData.csv
4309
reading E:\projects\stock data\2019May.zip : 2019May/20190509_OData.csv
4310
reading E:\projects\stock data\2019May.zip : 2019May/20190510_OData.csv
4312
reading E:\projects\stock data\2019May.zip : 2019May/20190513_OData.csv
4310
reading E:\projects\stock data\2019May.zip : 2019May/20190514_OData.csv
4313
reading E:\projects\stock data\2019May.zip : 2019May/20190515_OData.csv
4307
reading E:\projects\stock data\2019May.zip : 2019May/20190516_OData.csv
4307
reading E:\projects\stock data\2019May.zip : 2019May/20190517_OData.csv
4306
reading E:\projects\stock data\2019May.zip : 2019May/20190520_OData.csv

4263
reading E:\projects\stock data\D_2019June.zip : D_2019June/D_20190619_OData.csv
4262
reading E:\projects\stock data\D_2019June.zip : D_2019June/D_20190620_OData.csv
4262
reading E:\projects\stock data\D_2019June.zip : D_2019June/D_20190624_OData.csv
4260
reading E:\projects\stock data\D_2019June.zip : D_2019June/D_20190625_OData.csv
4260
reading E:\projects\stock data\D_2019June.zip : D_2019June/D_20190626_OData.csv
4261
reading E:\projects\stock data\D_2019June.zip : D_2019June/D_20190627_OData.csv
4264
reading E:\projects\stock data\D_2019June.zip : D_2019June/D_20190628_OData.csv
4273
reading E:\projects\stock data\D_2019November.zip : D_20191101_OData.csv
4254
reading E:\projects\stock data\D_2019November.zip : D_20191104_OData.csv
4250
reading E:\projects\stock data\D_2019November.zip : D_20191105_OData.csv
4253
reading E:\projects\stock data\D_2019November.zip : D_20191106_OData.csv
4250
reading E:\projects\stock data\D_2019November.zip : D_20191107_OData.csv
4253
reading E:

4228
reading E:\projects\stock data\D_2020August.zip : D_20200803_OData1.csv
2188
reading E:\projects\stock data\D_2020August.zip : D_20200803_OData2.csv
2073
reading E:\projects\stock data\D_2020August.zip : D_20200804_OData1.csv
2188
reading E:\projects\stock data\D_2020August.zip : D_20200804_OData2.csv
2072
reading E:\projects\stock data\D_2020August.zip : D_20200805_OData1.csv
2189
reading E:\projects\stock data\D_2020August.zip : D_20200805_OData2.csv
2075
reading E:\projects\stock data\D_2020August.zip : D_20200806_OData1.csv
2189
reading E:\projects\stock data\D_2020August.zip : D_20200806_OData2.csv
2075
reading E:\projects\stock data\D_2020August.zip : D_20200807_OData1.csv
2189
reading E:\projects\stock data\D_2020August.zip : D_20200807_OData2.csv
2074
reading E:\projects\stock data\D_2020August.zip : D_20200810_OData1.csv
2197
reading E:\projects\stock data\D_2020August.zip : D_20200810_OData2.csv
2101
reading E:\projects\stock data\D_2020August.zip : D_20200811_OData1.csv

2081
reading E:\projects\stock data\D_2020July.zip : D_20200716_OData1.csv
2166
reading E:\projects\stock data\D_2020July.zip : D_20200716_OData2.csv
2081
reading E:\projects\stock data\D_2020July.zip : D_20200717_OData1.csv
2166
reading E:\projects\stock data\D_2020July.zip : D_20200717_OData2.csv
2074
reading E:\projects\stock data\D_2020July.zip : D_20200720_OData1.csv
2170
reading E:\projects\stock data\D_2020July.zip : D_20200720_OData2.csv
2076
reading E:\projects\stock data\D_2020July.zip : D_20200721_OData1.csv
2170
reading E:\projects\stock data\D_2020July.zip : D_20200721_OData2.csv
2076
reading E:\projects\stock data\D_2020July.zip : D_20200722_OData1.csv
2172
reading E:\projects\stock data\D_2020July.zip : D_20200722_OData2.csv
2076
reading E:\projects\stock data\D_2020July.zip : D_20200723_OData1.csv
2172
reading E:\projects\stock data\D_2020July.zip : D_20200723_OData2.csv
2076
reading E:\projects\stock data\D_2020July.zip : D_20200724_OData1.csv
2171
reading E:\projects\

2201
reading E:\projects\stock data\D_2020September.zip : 2020September/D_20200902_OData2.csv
2116
reading E:\projects\stock data\D_2020September.zip : 2020September/D_20200903_OData1.csv
2201
reading E:\projects\stock data\D_2020September.zip : 2020September/D_20200903_OData2.csv
2115
reading E:\projects\stock data\D_2020September.zip : 2020September/D_20200904_OData1.csv
2200
reading E:\projects\stock data\D_2020September.zip : 2020September/D_20200904_OData2.csv
2116
reading E:\projects\stock data\D_2020September.zip : 2020September/D_20200907_OData1.csv
2200
reading E:\projects\stock data\D_2020September.zip : 2020September/D_20200907_OData2.csv
2115
reading E:\projects\stock data\D_2020September.zip : 2020September/D_20200908_OData1.csv
2200
reading E:\projects\stock data\D_2020September.zip : 2020September/D_20200908_OData2.csv
2115
reading E:\projects\stock data\D_2020September.zip : 2020September/D_20200909_OData1.csv
2201
reading E:\projects\stock data\D_2020September.zip : 20

In [4]:
df = pd.concat(df_parts)

In [19]:
df = df[~df['Symbol'].isna()]

In [20]:
df.to_csv('E:/projects/stock data/only_stock_prices.csv', index=False)

# find bad data
These are stock with apperant bad data. We will filter them out

In [59]:
all_symbols = set(df['Symbol'].unique())

In [60]:
zero_counts = {}
daily_spikes = {}
for symbol in all_symbols:
    print(f'testing {symbol}')
    df_symbol = df[df['Symbol'] == symbol]
    
    # test for too many zeros in price
    zero_counts[symbol] = sum(df_symbol['Price'] == 0) / len(df_symbol)
    if zero_counts[symbol]  > 0:
        print('  got suspicious zeros in stock price')
        
    df_diff = df_symbol[['Price']].pct_change()
    daily_spikes[symbol] = (df_diff > 20).sum().item()
    if daily_spikes[symbol] > 0:
        print('  got suspicious daily spike')


testing ZSL
testing YCL
  got suspicious zeros in stock price
  got suspicious daily spike
testing KCI
  got suspicious daily spike
testing CRWS
testing PMI
testing ACS
testing KRNY
testing ARNC2
testing CIEQ1
testing GSKY
testing MCY
testing SPXS
testing PRO
testing OUTR
testing EBRYY
testing TPTX
testing DXL
testing MWY
  got suspicious daily spike
testing XLFS
testing AKRX
testing TLO
testing TNY
testing TFSL
testing SNHY
testing MWW
testing BL
testing BUCY
testing AGI
  got suspicious daily spike
testing CPA
testing LIT
testing TTPY
testing WLL1
testing ATN
  got suspicious zeros in stock price
  got suspicious daily spike
testing PAA
  got suspicious daily spike
testing ADRO
testing DMND
testing FTR1
testing PCOM
testing DTD
testing ASVI
testing LRGF
testing SRL
testing ABRX
testing DBD
  got suspicious daily spike
testing ARLZ
  got suspicious zeros in stock price
  got suspicious daily spike
testing LDK
testing SMT
testing NMGA
testing SPU
testing PLXT
testing AKG
testing PCG
te

testing VALE.P
testing SANW
testing CXRX
testing WSTC
testing BVX
testing RYB
testing PENN
testing MAS
  got suspicious daily spike
testing HSX
testing SMMT
testing GMAB
testing CODE
  got suspicious daily spike
testing PNG
testing NITE
testing DBTK
testing CLNS2
testing DFRG
testing IART
  got suspicious daily spike
testing WLTGQ
testing DWAS
testing BETR
  got suspicious zeros in stock price
  got suspicious daily spike
testing OUNZ
testing BMR
testing HBAN
testing CSTL
testing CBPX
testing VRM
testing SM
testing RYI
  got suspicious zeros in stock price
  got suspicious daily spike
testing ELS
  got suspicious daily spike
testing SRR
  got suspicious daily spike
testing BCRX
testing ON
testing OSW
testing FNBC
testing VIIX
testing NUAN1
testing FOE
testing DBS
  got suspicious zeros in stock price
  got suspicious daily spike
testing FTGC
testing FRE
  got suspicious daily spike
testing VPL
testing SBRA1
testing ALQA1
testing ASAN
testing TRAD
testing CVE
testing IDEV
testing MASI
t

  got suspicious daily spike
testing HOGS
testing AGG
testing MOMO
testing HTLD
testing VRS
  got suspicious daily spike
testing PAYC
testing NJR
testing JNK
testing DEIX
testing FIF
testing SHOS
testing YUK
testing PRV
testing IFLO
testing CJHBQ
testing REGI
testing TPVG
testing ETM
testing WHQ
  got suspicious zeros in stock price
  got suspicious daily spike
testing BFAM
testing ANR
testing ENIC
testing FHN
testing AMB
testing KATE
testing PHR
testing VMEM
testing UTHR
testing KCAP
  got suspicious zeros in stock price
  got suspicious daily spike
testing TGI
testing GMEB
testing LIFC
testing K
testing HUI
  got suspicious zeros in stock price
  got suspicious daily spike
testing LDN
testing LXP
testing FRTA
testing AHH
testing STAR
testing XCOOQ
testing MAIN
testing NVLNF
testing QSII
  got suspicious zeros in stock price
testing PETS
testing AIG
  got suspicious daily spike
testing SXE
testing KBR
testing GMS
testing IPS
  got suspicious zeros in stock price
  got suspicious daily

  got suspicious zeros in stock price
  got suspicious daily spike
testing VTAL
testing HRZN
testing AXL
  got suspicious daily spike
testing PES
testing AGR
  got suspicious zeros in stock price
  got suspicious daily spike
testing JCOM
testing ABP
  got suspicious zeros in stock price
  got suspicious daily spike
testing VIXY1
testing XL
  got suspicious zeros in stock price
testing SWIR1
testing SELB
testing MAXR1
testing NWSA
testing CHL
testing NBR
  got suspicious daily spike
testing NTTYY
testing THX
  got suspicious zeros in stock price
  got suspicious daily spike
testing PRMW
testing LAMR
testing PAY
  got suspicious zeros in stock price
  got suspicious daily spike
testing ALEC
testing FXE
testing LINU
testing FFCH
testing SAA
testing ROYT
testing SPHD
testing SFYW
testing DD1
testing AXCA
testing ECST
testing CVY
testing XXIA
testing ELRC
testing ARLP
testing TOM
  got suspicious zeros in stock price
  got suspicious daily spike
testing PCYG
testing DOMR
testing EPR
testing

testing CYT
testing HCAC
testing GSLI
testing SHLM
  got suspicious zeros in stock price
  got suspicious daily spike
testing SNR
testing PNTP
testing GM
  got suspicious daily spike
testing CFC
  got suspicious zeros in stock price
  got suspicious daily spike
testing ROP
  got suspicious daily spike
testing ANSI
testing USM
  got suspicious daily spike
testing APLP
  got suspicious zeros in stock price
testing WMIH1
  got suspicious zeros in stock price
testing CCC
  got suspicious zeros in stock price
  got suspicious daily spike
testing FRAC
testing XLRE
testing OCNW
testing EE
  got suspicious daily spike
testing BRSS
testing UAMY
testing RNF
testing DUST
  got suspicious daily spike
testing RESN
testing NCRX
testing WHXT
testing WTR
  got suspicious daily spike
testing HRL
testing NMR
  got suspicious daily spike
testing ZUMZ
testing GG1
testing PMTCD
testing PXH
testing ALA
testing SERV
testing ACLID
testing MIRKQ
testing ITRN
testing USQ
testing EPU
testing GNTX
testing INT
  g

...

testing AVYA
testing ARTX
testing SLW
testing BLTI
testing CYX
  got suspicious daily spike
testing GA
testing SERV1
testing CDIS
testing EWC
testing MRX
  got suspicious daily spike
testing BW1
testing BR
  got suspicious zeros in stock price
  got suspicious daily spike
testing CHAPQ
testing TLND
testing SEMI
testing RBYCF
testing THS
  got suspicious daily spike
testing CCEP
  got suspicious zeros in stock price
  got suspicious daily spike
testing F
testing SVVS
testing IEV
testing XMSR
testing DRIP
  got suspicious daily spike
testing TLMR
testing WMI
  got suspicious daily spike
testing BNZ
testing VIEWF
testing RTKHQ
testing EAS
  got suspicious zeros in stock price
  got suspicious daily spike
testing CERS
testing SPXS1
testing APHT
testing EXLS
  got suspicious daily spike
testing UPWK
testing MFS
testing XAN
testing DST
  got suspicious zeros in stock price
testing AMCR
testing FPAC
testing ZHNE
testing GKE
  got suspicious zeros in stock price
  got suspicious daily spike
te

testing SIGM
  got suspicious daily spike
testing RHEO
testing TWLO
testing POY
testing TWP
  got suspicious daily spike
testing LABS
testing ACHI
testing BPT
testing CIVB
  got suspicious daily spike
testing LCAPA
  got suspicious daily spike
testing CACH2
testing LLEXQ
testing CAK
testing RALY
testing SUZ1
testing NX
  got suspicious daily spike
testing WEBX
testing SIGA
testing EDS
  got suspicious zeros in stock price
  got suspicious daily spike
testing NANR
testing MTGE
  got suspicious zeros in stock price
testing FORM
testing CDSCY
testing LEND
testing FXA
testing XON
testing CNSL
testing GOLD
testing CLDPQ
testing VRNS
testing SPWR
testing PIN
testing PPA
  got suspicious daily spike
testing HK2
testing LII
  got suspicious daily spike
testing GEOKQ
testing MBT
testing MIDZ1
testing FLNT1
testing FUQI
testing SNOW
testing SHEN
  got suspicious daily spike
testing PWT
  got suspicious daily spike
testing PWER
testing ADAP
testing TTEK
testing DBJP
testing XLV
testing XNCR
testi

testing ASMLD
testing ALSC
testing PXN
testing WSH
  got suspicious daily spike
testing IRF
testing NPD
testing AVB
  got suspicious daily spike
testing INX
testing YONG
testing UMH
testing NYMX
testing MANH
testing IT
  got suspicious daily spike
testing ARCP
testing EMMS
  got suspicious daily spike
testing SRI
testing RESI
testing DLX
testing DRTX
testing EZA
testing PKD1
testing MYRX
  got suspicious daily spike
testing TLSA
testing ASFI
testing HPE2
testing AHO
  got suspicious zeros in stock price
  got suspicious daily spike
testing OREX1
testing PXD
testing HOT
testing HAFC
testing ERN
  got suspicious zeros in stock price
testing FNFG
testing APO
testing RSHC2
testing KT
testing VSS
testing SVO
testing FBNK
  got suspicious zeros in stock price
testing VXRT1
testing MSCC
  got suspicious zeros in stock price
  got suspicious daily spike
testing BPL
testing CRS
testing FSIC
  got suspicious zeros in stock price
  got suspicious daily spike
testing SGMO
testing ATHR
testing CRNT

testing KFRC
testing KAMN
testing LNR
testing ATGE
testing FLR
testing MWN
testing IILG
testing CTRN
  got suspicious daily spike
testing H
  got suspicious zeros in stock price
  got suspicious daily spike
testing BAL
  got suspicious zeros in stock price
  got suspicious daily spike
testing AVAN
testing DRYR
testing EEB
testing PLLL
testing KPTI
testing UGI1
testing ICLK
testing SORL
testing CNXN
testing ROCK
testing KMRT
testing GBCI
  got suspicious daily spike
testing MOVE
testing ANY1
testing NTUS
testing MKSI
testing AAUKY
testing KONAQ
  got suspicious zeros in stock price
  got suspicious daily spike
testing VELT
testing LEJU
testing HDYN2
testing RGDO
testing EP
testing XSPA1
testing BKS
testing PIRS
testing OPWV
testing TROV1
testing CFI
testing AOB
  got suspicious daily spike
testing PCYO
  got suspicious daily spike
testing CBSH
testing NMRD
testing XHB
testing CSA
  got suspicious daily spike
testing XDA
testing RIG
testing ACIA
testing OXFD
testing USHY1
testing DNR
tes

  got suspicious daily spike
testing DTN
testing CRDN
testing SMI
  got suspicious daily spike
testing VLP
  got suspicious zeros in stock price
testing RTRX
testing LMNX
testing BPFH
testing DCMYY
  got suspicious zeros in stock price
  got suspicious daily spike
testing IEF
testing DIS1
testing PSG
testing AIT
testing HCCI
  got suspicious daily spike
testing JLL1
testing COPX
testing HCN
  got suspicious zeros in stock price
  got suspicious daily spike
testing STTSY
testing DTSI
testing NNOX
testing CTL1
testing SLXA
testing TXCO
testing ICBC
testing RNST
testing WINN
testing PZG
testing IXYS
  got suspicious zeros in stock price
testing UBS
testing NDN
  got suspicious daily spike
testing CHKP
testing SIMGE
testing MDGN
testing MESA
  got suspicious daily spike
testing WWAV/B
testing SZMK
testing LEE
  got suspicious daily spike
testing UHS
  got suspicious daily spike
testing FPRX
testing IIT
testing CNE
  got suspicious zeros in stock price
  got suspicious daily spike
testing L

In [91]:
bad_stocks = {ticker for ticker, zero_p in zero_counts.items() if zero_p > 0} | \
    {ticker for ticker, spike_n in daily_spikes.items() if spike_n > 0}

In [93]:
len(bad_stocks) / len(all_symbols)

0.18443126022913256

In [96]:
with open('E:/projects/stock data/bad_stocks.txt', 'w') as f:
    f.write(','.join(bad_stocks))